In [151]:
import requests
import json
import pandas as pd

snapshot_api = 'https://hub.snapshot.org/graphql'
covalent_api = 'https://api.covalenthq.com/v1/'
covalent_key = 'ckey_1c9890fa556c457ab9ff050c2f4'

ens_token_addr = '0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198' # for testing purpose

latest_block = 14548652
max_r_blocks = 1000000

# 1. Snapshot API
### (1) Query Proposals

In [65]:
GET_PROPOSALS = """
query ($first:Int!, $skip:Int!, $space_in:[String]!) {
  proposals (
    first: $first,
    skip: $skip,
    where: {
      space_in: $space_in,
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    space {
      id
      name
    }
  }
}
"""

In [67]:
r = requests.post(snapshot_api, json={
    'query': GET_PROPOSALS, 
    'variables': {
        'first': 1000,
        'skip': 0,
        'space_in': ['ens.eth']
    }
})
print('Status:', r.status_code)
# print(r.text)
# Load to JSON
json_data = json.loads(r.text)

Status: 200


In [68]:
# total # of proposals
proposals = json_data['data']['proposals']
len(proposals)

22

In [69]:
# Ids for each proposal
p0 = proposals[0]
p0.keys()
# dict_keys(['id', 'title', 'body', 'choices', 'start', 'end', 'snapshot', 'state', 'author', 'space'])

dict_keys(['id', 'title', 'body', 'choices', 'start', 'end', 'snapshot', 'state', 'author', 'space'])

In [70]:
for i in p0.keys():
    print('\n', i,':\n',p0[i])


 id :
 0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16

 title :
 [EP10] [Executable] Fund a DAO-Governed Identity Server

 body :
 Authors: [Gregory Rocco](https://github.com/obstropolos), [Wayne Chang](https://github.com/wyc)

# Description
This proposal is for the funding and establishment of a community-run OIDC Identity Provider Server for Sign-In with Ethereum, maintained by Spruce.

Temperature check originally from [this post](https://discuss.ens.domains/t/a-credibly-neutral-sign-in-with-ethereum-identity-provider-server/11001)

# Abstract
In our research, we found that many services wanted to integrate the Sign-In with Ethereum workflow but did not have the ability to add new passwordless authentication methods to their installations.

We also learned that most services already support OpenID Connect, and were open to adding a new Identity Provider that supported the SIWE workflow. By meeting those services where they are today, we can provide a pragmatic s

### 2. Get Votes

In [60]:
GET_VOTES = """query ($first:Int!, $skip:Int!, $proposal:String!) {
  votes (
    first: $first
    skip: $skip
    where: {
      proposal: $proposal
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    voter
    created
    proposal {
      id
    }
    choice
    space {
      id
    }
  }
}
"""

In [71]:
r = requests.post(snapshot_api, json={
    'query': GET_VOTES, 
    'variables': {
        'first': 1000,
        'skip': 0,
        'proposal': '0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16'
    }
})
print('Status:', r.status_code)
# print(r.text)
# Load to JSON
json_data = json.loads(r.text)

Status: 200


In [76]:
# total # of votes
votes = json_data['data']['votes']
len(votes)

445

In [79]:
v0 = votes[0]
for i in v0.keys():
    print('\n', i, ':\n', v0[i])


 id :
 0xb5502554b55912be5be38cbd7c3567decd324f8734a28e567848010c14028f58

 voter :
 0x6eB0F29b8Fc7A29E517BA56Dd79287B3CCaC64f3

 created :
 1647809547

 proposal :
 {'id': '0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16'}

 choice :
 1

 space :
 {'id': 'ens.eth'}


# 2. Covalent

In [186]:
# Helper functions

def request_covalent(url):
    r = requests.get(url)
    if r.status_code == 200:
        print('request succeeded!')
        return json.loads(r.text)
    else:
        print('Request Error:', r)
def r_token_holder_url(network_id, proposal_id, page_size, page_number):
    r_url = covalent_api + str(network_id) + '/tokens/' + str(proposal_id) + '/token_holders/?quote-currency=USD&format=JSON&page-size=' + str(page_size) + '&page-number=' + str(page_number) + '&key=' + covalent_key
    return request_covalent(r_url)

def r_log_events_url(network_id, address, page_size=10000, start_block=latest_block-max_r_blocks, end_block=latest_block):
    r_url = covalent_api + str(network_id) + '/events/address/' + str(address) + '/?starting-block=' + str(start_block) + '&ending-block=' + str(end_block) + '&page-size=' + str(page_size) + '&key=' + covalent_key
    print(r_url)
    return request_covalent(r_url)

def r_token_balance(network_id, address):
    r_url = covalent_api + str(network_id) + '/address/' + str(address) + '/balances_v2/?quote-currency=USD&format=JSON&nft=false&no-nft-fetch=false&key=' + covalent_key
    return request_covalent(r_url)

def r_portfolio_hist_val(network_id, address):
    r_url = covalent_api + str(network_id) + '/address/' + str(address) + '/portfolio_v2/?&key=' + covalent_key
    return request_covalent(r_url)

### (1) Get all token holders

In [126]:
r = r_token_holder_url(1, ens_token_addr, 10000, 0)
print('Status:', r.status_code)
# Load to JSON
json_data = json.loads(r.text)

Status: 200


In [120]:
json_data['data'].keys()

dict_keys(['updated_at', 'items', 'pagination'])

In [119]:
json_data['data']['pagination']

{'has_more': False, 'page_number': 0, 'page_size': 10000, 'total_count': 4872}

In [121]:
len(json_data['data']['items'])

4872

In [115]:
request_url = r_token_holder_url(1, ens_token_addr, 1000, 1)
print(request_url)
r = requests.get(request_url)
print('Status:', r.status_code)
# Load to JSON
json_data = json.loads(r.text)

https://api.covalenthq.com/v1/1/tokens/0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198/token_holders/?quote-currency=USD&format=JSON&page-size=1000&page-number=1&key=ckey_1c9890fa556c457ab9ff050c2f4
Status: 200


In [123]:
json_data['data']['items'][0]

{'contract_decimals': 18,
 'contract_name': 'Bankless Token',
 'contract_ticker_symbol': 'BANK',
 'contract_address': '0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198',
 'supports_erc': None,
 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198.png',
 'address': '0xdc351121342d0d25c80203201386323c1dcc7365',
 'balance': '295294492755974358753371570',
 'total_supply': '1000000000000000000000000000',
 'block_height': 14548652}

### (2) Get log events by contract address

In [166]:
r = r_log_events_url(1, '0xbec3de5b14902c660bd2c7efd2f259998424cc24',10000, latest_block - max_r_blocks, latest_block)

https://api.covalenthq.com/v1/1/events/address/0xbec3de5b14902c660bd2c7efd2f259998424cc24/?starting-block=13548652&ending-block=14548652&page-size=10000&key=ckey_1c9890fa556c457ab9ff050c2f4
request succeeded!


In [168]:
r.keys()
# dict_keys(['data', 'error', 'error_message', 'error_code'])

r['data'].keys()
# dict_keys(['updated_at', 'items', 'pagination'])

len(r['data']['items'])
# 195

r['data']['pagination']
# {'has_more': None, 'page_number': 0, 'page_size': 10000, 'total_count': None}

data = r['data']['items']

In [170]:
data[0]

{'block_signed_at': '2021-11-04T14:11:46Z',
 'block_height': 13550680,
 'tx_offset': 180,
 'log_offset': 223,
 'tx_hash': '0x24b2038efe329b33f2ef6869abd8e7d8fc768429540680df83a21a587665a063',
 'raw_log_topics': ['0xb8e138887d0aa13bab447e82de9d5c1777041ecd21ca36ba824ff1e6c07ddda4',
  '0x000000000000000000000000eeebdeaec2c87ee38fa8aa3a148f49a87990d30c'],
 'sender_contract_decimals': 0,
 'sender_name': 'BabylonGovernor',
 'sender_contract_ticker_symbol': None,
 'sender_address': '0xbec3de5b14902c660bd2c7efd2f259998424cc24',
 'sender_address_label': None,
 'sender_logo_url': 'https://logos.covalenthq.com/tokens/1/0xbec3de5b14902c660bd2c7efd2f259998424cc24.png',
 'raw_log_data': '0xdf8c9ac4d8ae4b808f95338a8d5b0ae478aed8de8b7b9c6d95ece1bfc0d6af5c0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000077432217e68360000000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000

### (3) Token balance given address

In [175]:
r = r_token_balance(1,'0x31f8cc382c9898b273eff4e0b7626a6987c846e8')

request succeeded!


In [183]:
r.keys()
# dict_keys(['data', 'error', 'error_message', 'error_code'])

r['data'].keys()
# dict_keys(['address', 'updated_at', 'next_update_at', 'quote_currency', 'chain_id', 'items', 'pagination'])

len(r['data']['items'])
# 56

len(r['data']['quote_currency'])
# 3 (I think it is USD)

r['data']['pagination']
# empty

data = r['data']['items']

In [184]:
data[0]

{'contract_decimals': 18,
 'contract_name': 'Discord DAO',
 'contract_ticker_symbol': 'DISCO',
 'contract_address': '0xf65c59addd68bc186f2b0e68f4343e26666bacae',
 'supports_erc': ['erc20'],
 'logo_url': 'https://logos.covalenthq.com/tokens/1/0xf65c59addd68bc186f2b0e68f4343e26666bacae.png',
 'last_transferred_at': '2022-02-03T19:44:53Z',
 'type': 'cryptocurrency',
 'balance': '19788401086904410237744296385932',
 'balance_24h': '19788401086904410237744296385932',
 'quote_rate': 1.269996e-05,
 'quote_rate_24h': None,
 'quote': 251311904.0,
 'quote_24h': None,
 'nft_data': None}

### (4) Historical portfolio value

In [188]:
r = r_portfolio_hist_val(1, '0x31f8cc382c9898b273eff4e0b7626a6987c846e8')

request succeeded!


In [197]:
r.keys()
# dict_keys(['data', 'error', 'error_message', 'error_code'])

r['data'].keys()
# dict_keys(['address', 'updated_at', 'next_update_at', 'quote_currency', 'chain_id', 'items', 'pagination'])

len(r['data']['items'])
# 56

len(r['data']['quote_currency'])
# USD

r['data']['pagination']
# empty

data = r['data']['items']

In [198]:
data[0]

{'contract_decimals': 18,
 'contract_name': 'Discord DAO',
 'contract_ticker_symbol': 'DISCO',
 'contract_address': '0xf65c59addd68bc186f2b0e68f4343e26666bacae',
 'supports_erc': None,
 'logo_url': 'https://logos.covalenthq.com/tokens/1/0xf65c59addd68bc186f2b0e68f4343e26666bacae.png',
 'holdings': [{'timestamp': '2022-04-09T00:00:00Z',
   'quote_rate': 1.269996e-05,
   'open': {'balance': '19788401086904410237744296385932',
    'quote': 251311904.0},
   'high': {'balance': '19788401086904410237744296385932',
    'quote': 251311904.0},
   'low': {'balance': '19788401086904410237744296385932',
    'quote': 251311904.0},
   'close': {'balance': '19788401086904410237744296385932',
    'quote': 251311904.0}},
  {'timestamp': '2022-04-08T00:00:00Z',
   'quote_rate': None,
   'open': {'balance': '19788401086904410237744296385932', 'quote': None},
   'high': {'balance': '19788401086904410237744296385932', 'quote': None},
   'low': {'balance': '19788401086904410237744296385932', 'quote': None},